# LaQuacco 🍅🍅🍅

## Laboratory Quality Control v1.0

### Module Imports

In [ ]:
import os
import time
import matplotlib.pyplot as plt
import numpy as np
import laquacco as laq  # custom functions

### User Input

In [ ]:
# define search pattern and path for positive controls:
# statistics will be used for later quality control
control_dir = r"./tests/Polaris"  # directory string
control_ext = "*.tif"  # positive search pattern
control_exc = ""  # negative search pattern
control_sub = True  # recursive search

# define search pattern and path for dataset samples
# run quality control for each channel
sample_dir = r"./tests/Polaris"
sample_ext = "*.tif"
sample_exc = ""
sample_sub = True

### Check Dataset

In [ ]:
# list positive controls
controls = sorted(
    laq.get_files(
        path=control_dir,
        pat=control_ext,
        anti=control_exc,
        recurse=control_sub,
    ),
    key=str.lower,
)
controls_len = len(controls)
print(f"Controls: {controls_len}\n\t{os.path.abspath(control_dir)}")
for control in controls:
    print(f"\t {control.replace(control_dir, '.')}")
print()

# list dataset samples
samples = sorted(
    laq.get_files(
        path=sample_dir,
        pat=sample_ext,
        anti=sample_exc,
        recurse=sample_sub,
    ),
    key=str.lower,
)
samples_len = len(samples)
print(f"Controls: {samples_len}\n\t{os.path.abspath(sample_dir)}")
for sample in samples:
    print(f"\t {sample.replace(sample_dir, '.')}")

### Sample Dataset

In [ ]:
# get control statistics
controls_img_data = {}
start = time.time()
for count, control in enumerate(controls, start=1):
    print(
        f"CONTROL: [{count}/{controls_len}]  {os.path.basename(control)}",
        end="",
        flush=True,
    )
    controls_img_data[control] = laq.stats_img_data(
        laq.get_tiff(control)
    )  # signal above zero
    print(f"\t(-{laq.get_time_left(start, count, controls_len)})")
print()

# list control channel names (by name)
chans = dict()
for img_data in controls_img_data.values():
    for chan in img_data:
        chans[chan] = None
chans = list(chans.keys())
chans.remove("metadata")
chans = sorted(chans, key=str.lower)

# get control statistics
control_stats = {chan: (np.nan, np.nan) for chan in chans}
for chan in chans:
    print(f"{chan}:")
    control_mean = laq.get_chan_data(controls_img_data, chan, "mean")
    control_minmax = laq.get_chan_data(controls_img_data, chan, "minmax", 2)
    control_stats[chan] = (
        np.nanmean(control_mean),
        np.nanmean(control_minmax[:, 0]),
        np.nanmean(control_minmax[:, 1]),
    )
    print(
        f"\tMaximum: {control_stats[chan][2]:7.1f} (mean) {np.nanstd(control_minmax[:, 1]):7.1f} (std)\n",
        f"\tMean:    {control_stats[chan][0]:7.1f} (mean) {np.nanstd(control_mean):7.1f} (std)\n",
        f"\tMinimum: {control_stats[chan][1]:7.1f} (mean) {np.nanstd(control_minmax[:, 0]):7.1f} (std)\n",
    )

# get sample statistics
samples_img_data = {}
start = time.time()
for count, sample in enumerate(samples, start=1):
    print(
        f"SAMPLE: [{count}/{samples_len}]  {os.path.basename(sample)}",
        end="",
        flush=True,
    )
    samples_img_data[sample] = laq.stats_img_data(laq.get_tiff(sample), control_stats)
    print(f"\t(-{laq.get_time_left(start, count, samples_len)})")
print()

# sort samples by time stamp
samples_img_data = dict(
    sorted(samples_img_data.items(), key=lambda v: v[1]["metadata"]["date_time"])
)
samples = [sample for sample in samples_img_data.keys()]

### Graphics Settings

In [ ]:
# prepare figure dimensions
dpi = plt.rcParams["figure.dpi"]
min_pixw, min_pixh = 1600, 1200
min_width, min_height = min_pixw / dpi, min_pixh / dpi
plt.rcParams["figure.figsize"] = [min_width, min_height]

### Data Plots I - Distribution Chart

In [ ]:
print("Plotting distribution chart...\n")

# get sample means
sample_means = {}
fig, ax = plt.subplots()
for c, chan in enumerate(chans):
    sample_means[chan] = laq.get_chan_data(samples_img_data, chan, "mean")

# prepare variables for plotting
x_vals = range(1, len(chans) + 1)
y_vals = [y for y in sample_means.values()]

# create violin plot for all channels
ax.set_xticks(x_vals, labels=chans, rotation=90)
vp = ax.violinplot(y_vals, showmeans=True, showextrema=False)
for p in vp["bodies"]:
    p.set_facecolor("black")
    p.set_edgecolor("black")
for m in ["cmeans"]:
    vp[m].set_edgecolor("dimgray")

# adjust axes
plt.minorticks_on()
plt.ylim(bottom=0.0)

# add grid
plt.grid(color='gray', linewidth=0.25)

# add legend
legend = plt.legend(
    [vp["bodies"][0]],
    ["mean"],
)

# get axes limits
y_min, y_max = plt.gca().get_ylim()

# show plot
plt.tight_layout()
plt.show()


### Data Plots II - Levey-Jennings Charts

In [ ]:
print("Plotting Levey-Jennings charts...")

# one plot per channel
chan_limits = {}
for c, chan in enumerate(chans):
    print(f"\n{chan}:")

    # get image statistics
    chan_mean = np.nanmean(sample_means[chan])
    chan_stdevs = laq.get_chan_data(samples_img_data, chan, "stdev")
    chan_stdev = np.nanstd(sample_means[chan])
    chan_stderrs = laq.get_chan_data(samples_img_data, chan, "stderr")

    # prepare variables for plotting
    x_vals = range(1, samples_len + 1)
    y_vals = sample_means[chan]
    e_vals = chan_stderrs

    # plot error bars
    plt.errorbar(
        x_vals,
        y_vals,
        yerr=e_vals,
        fmt="o-",
        linewidth=1,
        markersize=3,
        color="black",
        label=chan,
    )

    # adjust axes
    plt.minorticks_on()
    plt.ylim(bottom=y_min, top=y_max)

    # add legend
    plt.legend()

    # add grid
    plt.grid(color='gray', linewidth=0.25)

    # add channel mean
    plt.axhline(y=chan_mean, color="black", linestyle="dashed")

    # add channel limits
    chan_limits_keys = [("p2stdev", "m2stdev"), ("p1stdev", "m1stdev")]
    chan_limits[chan] = {
        chan_limits_keys[0][0]: chan_mean + 2.0 * chan_stdev,
        chan_limits_keys[1][0]: chan_mean + 1.0 * chan_stdev,
        chan_limits_keys[1][1]: chan_mean - 1.0 * chan_stdev,
        chan_limits_keys[0][1]: chan_mean - 2.0 * chan_stdev,
    }
    for upper, lower in chan_limits_keys:
        plt.fill_between(
            x_vals,
            chan_limits[chan][upper],
            chan_limits[chan][lower],
            color="black",
            alpha=0.1,
        )

    # show plot
    plt.tight_layout()
    plt.show()

# print legend for x axes
print("Samples:")
for position, sample in enumerate(samples, start=1):
    print(f"\t{position} = {os.path.basename(sample)}")

"""
"""

print("Listing extreme values...")

for c, chan in enumerate(chans):
    print(f"\n{chan}:")

    for m, mean in enumerate(sample_means[chan]):
        if mean > chan_limits[chan]["p2stdev"]:
            print(f"\t▲▲ {os.path.basename(samples[m])}")
        elif mean > chan_limits[chan]["p1stdev"]:
            print(f"\t▲  {os.path.basename(samples[m])}")
        elif mean < chan_limits[chan]["m2stdev"]:
            print(f"\t▼▼ {os.path.basename(samples[m])}")
        elif mean < chan_limits[chan]["m1stdev"]:
            print(f"\t▼  {os.path.basename(samples[m])}")

### Data Plots II - Extreme Values

In [ ]:
print("Listing extreme values...")

for c, chan in enumerate(chans):
    print(f"\n{chan}:")

    for m, mean in enumerate(sample_means[chan]):
        if mean > chan_limits[chan]["p2stdev"]:
            print(f"\t▲▲ {os.path.basename(samples[m])}")
        elif mean > chan_limits[chan]["p1stdev"]:
            print(f"\t▲  {os.path.basename(samples[m])}")
        elif mean < chan_limits[chan]["m2stdev"]:
            print(f"\t▼▼ {os.path.basename(samples[m])}")
        elif mean < chan_limits[chan]["m1stdev"]:
            print(f"\t▼  {os.path.basename(samples[m])}")

### Data Plots III - C-Band Charts

In [ ]:
print("Plotting C-Band charts...")

# one plot per channel
bands_limits = {}
chans_bands = {}
for c, chan in enumerate(chans):
    print(f"\n{chan}:")

    # add band to plot
    chans_bands[chan] = list(zip(*laq.get_chan_data(samples_img_data, chan, "bands", 4)))
    for lvl, bands in enumerate(chans_bands[chan]):

        # get image statistics
        bands_mean = np.nanmean(bands)
        bands_stdev = np.nanstd(bands)

        # prepare variables for plotting
        x_vals = range(1, samples_len + 1)
        y_vals = bands

        # plot error bars
        plt.errorbar(
            x_vals,
            y_vals,
            fmt="o-",
            linewidth=1,
            markersize=3,
            color="black",
            label=chan,
        )

        # add band mean
        plt.axhline(y=bands_mean, color="black", linestyle="dashed")

        # add band limits
        bands_limits[chan] = {}
        bands_limits_keys = [("p2stdev", "m2stdev"), ("p1stdev", "m1stdev")]
        bands_limits[chan][lvl] = {
            bands_limits_keys[0][0]: bands_mean + 2.0 * bands_stdev,
            bands_limits_keys[1][0]: bands_mean + 1.0 * bands_stdev,
            bands_limits_keys[1][1]: bands_mean - 1.0 * bands_stdev,
            bands_limits_keys[0][1]: bands_mean - 2.0 * bands_stdev,
        }
        for upper, lower in bands_limits_keys:
            plt.fill_between(
                x_vals,
                bands_limits[chan][lvl][upper],
                bands_limits[chan][lvl][lower],
                color="black",
                alpha=0.1,
            )

    # adjust axes
    plt.minorticks_on()
    plt.ylim(bottom=y_min)

    # add legend
    handles, labels = plt.gca().get_legend_handles_labels()
    plt.legend([handles[0]], [labels[0]])

    # add grid
    plt.grid(color='gray', linewidth=0.25)

    # add channel mean
    plt.axhline(y=np.nanmean(sample_means[chan]), color="black", linestyle="solid")

    # show plot
    plt.tight_layout()
    plt.show()

### Data Plots III - Signal-to-Noise Ratio

In [ ]:
print("Listing signal-to-noise ratios...")

chans_snrs = {}
for chan in chans:
    chans_snrs[chan] = []
    for lvl, bands in enumerate(chans_bands[chan]):
        chans_snrs[chan].append(tuple([band / band_0 for band, band_0 in zip(bands, list(chans_bands[chan])[0])]))

# print signal-to-noise ratios for bands
for c, chan in enumerate(chans):
    print(f"\n{chan}:")

    for position, sample in enumerate(samples):
        print(f"\t{os.path.basename(sample)}", end="")
        for snrs in chans_snrs[chan]:
            print(f"\t{snrs[position]:7.1f}", end="")
        print()